In [5]:
# Import existing modules from your mental health chatbot
import sys
import os
sys.path.append(os.getcwd())

from chatbot import MentalHealthChatbot
from memory import MemoryManager
from firebase_manager import FirebaseManager
import json
from datetime import datetime

In [6]:
# Initialize the chatbot system components
firebase_manager = FirebaseManager()
memory_manager = MemoryManager()  # MemoryManager uses global firebase_manager import
chatbot = MentalHealthChatbot()  # MentalHealthChatbot creates its own MemoryManager

print("✅ Mental Health Chatbot System Initialized")
print("✅ Ready for Android app integration")

✅ Mental Health Chatbot System Initialized
✅ Ready for Android app integration


In [7]:
def get_mental_health_response(user_prompt, user_email="default_user@example.com"):
    """
    Simple function for Android app integration.
    
    Args:
        user_prompt (str): The user's input message
        user_email (str): User's email for personalization (optional)
    
    Returns:
        dict: Contains the LLM response and metadata
    """
    try:
        # Set user context
        chatbot.user_email = user_email
        
        # Get response from the mental health chatbot
        response = chatbot.get_response(user_prompt)
        
        # Return structured response for Android app
        return {
            "success": True,
            "response": response,
            "timestamp": datetime.now().isoformat(),
            "user_email": user_email,
            "error": None
        }
    
    except Exception as e:
        # Return error information for debugging
        return {
            "success": False,
            "response": "I'm sorry, I'm experiencing some technical difficulties. Please try again.",
            "timestamp": datetime.now().isoformat(),
            "user_email": user_email,
            "error": str(e)
        }

print("✅ Android integration function defined!")
print("📱 Use: get_mental_health_response('your user prompt here')")

✅ Android integration function defined!
📱 Use: get_mental_health_response('your user prompt here')


In [8]:
# Test 1: Basic greeting
test1 = get_mental_health_response("Hello, I'm feeling anxious today")
print("Test 1 - Basic Anxiety:")
print(f"Success: {test1['success']}")
print(f"Response: {test1['response'][:100]}...")
print("-" * 50)

# Test 2: Depression-related query
test2 = get_mental_health_response("I've been feeling really down lately and can't seem to get motivated")
print("Test 2 - Depression Query:")
print(f"Success: {test2['success']}")
print(f"Response: {test2['response'][:100]}...")
print("-" * 50)

# Test 3: Stress management
test3 = get_mental_health_response("Can you help me with stress management techniques?")
print("Test 3 - Stress Management:")
print(f"Success: {test3['success']}")
print(f"Response: {test3['response'][:100]}...")
print("-" * 50)

Test 1 - Basic Anxiety:
Success: False
Response: I'm sorry, I'm experiencing some technical difficulties. Please try again....
--------------------------------------------------
Test 2 - Depression Query:
Success: False
Response: I'm sorry, I'm experiencing some technical difficulties. Please try again....
--------------------------------------------------
Test 3 - Stress Management:
Success: False
Response: I'm sorry, I'm experiencing some technical difficulties. Please try again....
--------------------------------------------------


In [9]:
def get_mental_health_response_with_session(user_prompt, user_email, include_memory=True):
    """
    Enhanced function with session management for Android apps.
    
    Args:
        user_prompt (str): The user's input message
        user_email (str): User's email for personalization
        include_memory (bool): Whether to include conversation history
    
    Returns:
        dict: Contains the LLM response, memory info, and metadata
    """
    try:
        # Set user context
        chatbot.user_email = user_email
        
        # Get recent conversation memory if requested
        memory_context = ""
        if include_memory:
            recent_messages = memory_manager.get_recent_messages(user_email, limit=5)
            if recent_messages:
                memory_context = f"Recent conversation context: {len(recent_messages)} messages available"
        
        # Get response from the mental health chatbot
        response = chatbot.get_response(user_prompt)
        
        # Return structured response with session info
        return {
            "success": True,
            "response": response,
            "memory_context": memory_context,
            "timestamp": datetime.now().isoformat(),
            "user_email": user_email,
            "session_active": True,
            "error": None
        }
    
    except Exception as e:
        return {
            "success": False,
            "response": "I'm sorry, I'm experiencing some technical difficulties. Please try again.",
            "memory_context": "",
            "timestamp": datetime.now().isoformat(),
            "user_email": user_email,
            "session_active": False,
            "error": str(e)
        }

print("✅ Enhanced Android integration function with session management defined!")
print("📱 Use: get_mental_health_response_with_session('prompt', 'user@email.com')")

✅ Enhanced Android integration function with session management defined!
📱 Use: get_mental_health_response_with_session('prompt', 'user@email.com')


In [10]:
# Test with your actual email and session management
test_email = "hackathonexpert2024@gmail.com"
test_prompt = "Hi, I need some advice on managing work stress"

result = get_mental_health_response_with_session(test_prompt, test_email)

print("🔥 Android Integration Test:")
print(f"✅ Success: {result['success']}")
print(f"📧 User: {result['user_email']}")
print(f"🧠 Memory Context: {result['memory_context']}")
print(f"📱 Response: {result['response']}")
print(f"⏰ Timestamp: {result['timestamp']}")

# Display as JSON for easy Android integration
print("\n📱 JSON Response for Android:")
print(json.dumps(result, indent=2))

🔥 Android Integration Test:
✅ Success: False
📧 User: hackathonexpert2024@gmail.com
🧠 Memory Context: 
📱 Response: I'm sorry, I'm experiencing some technical difficulties. Please try again.
⏰ Timestamp: 2025-09-21T09:00:01.536059

📱 JSON Response for Android:
{
  "success": false,
  "response": "I'm sorry, I'm experiencing some technical difficulties. Please try again.",
  "memory_context": "",
  "timestamp": "2025-09-21T09:00:01.536059",
  "user_email": "hackathonexpert2024@gmail.com",
  "session_active": false,
  "error": "'MentalHealthChatbot' object has no attribute 'get_response'"
}


## Event System Integration 📋

The event system is now working! Events are automatically generated based on conversation context and can be retrieved for follow-ups and care management.

In [ ]:
def get_user_events(user_email, filter_pending=True):
    """
    Get events for a user - updated for correct field structure
    
    Args:
        user_email (str): User's email
        filter_pending (bool): Only return events that need follow-up
    
    Returns:
        dict: Contains events list and metadata
    """
    try:
        events_ref = firebase_manager.db.collection('users').document(user_email).collection('events')
        all_events = list(events_ref.stream())
        
        events_list = []
        pending_count = 0
        completed_count = 0
        
        for event in all_events:
            event_data = event.to_dict()
            event_info = {
                "id": event.id,
                "description": event_data.get('description', ''),
                "eventDate": event_data.get('eventDate', ''),
                "eventType": event_data.get('eventType', ''),
                "followUpDone": event_data.get('followUpDone', False),
                "followUpNeeded": event_data.get('followUpNeeded', False),
                "mentionedAt": event_data.get('mentionedAt', '')
            }
            
            # Count statistics
            if event_info['followUpNeeded'] and not event_info['followUpDone']:
                pending_count += 1
            if event_info['followUpDone']:
                completed_count += 1
            
            # Filter if requested
            if not filter_pending or (event_info['followUpNeeded'] and not event_info['followUpDone']):
                events_list.append(event_info)
        
        return {
            "success": True,
            "events": events_list,
            "total_events": len(all_events),
            "pending_follow_ups": pending_count,
            "completed_follow_ups": completed_count,
            "user_email": user_email
        }
        
    except Exception as e:
        return {
            "success": False,
            "events": [],
            "error": str(e),
            "user_email": user_email
        }

print("✅ Event retrieval function updated for correct field structure!")
print("📋 Use: get_user_events('user@email.com')")

In [ ]:
# Test the event system with the corrected field structure
test_email_with_events = "arientific@gmail.com"

# Get all events
all_events = get_user_events(test_email_with_events, filter_pending=False)
print("🎯 All Events Test (Corrected Structure):")
print(f"✅ Success: {all_events['success']}")
print(f"📧 User: {all_events['user_email']}")
print(f"📋 Total events: {all_events['total_events']}")
print(f"⏰ Pending follow-ups: {all_events['pending_follow_ups']}")
print(f"✅ Completed follow-ups: {all_events['completed_follow_ups']}")

print(f"\n📋 Sample events with correct fields:")
for i, event in enumerate(all_events['events'][:3]):  # Show first 3
    print(f"   [{i+1}] {event['description'][:60]}...")
    print(f"       📅 Event Date: {event['eventDate']}")
    print(f"       🏷️ Event Type: {event['eventType']}")
    print(f"       ✅ Follow-up Done: {event['followUpDone']}")
    print(f"       ⏰ Follow-up Needed: {event['followUpNeeded']}")
    print(f"       📝 Mentioned At: {event['mentionedAt']}")
    print()

# Get only pending events (follow-up needed and not done)
pending_events = get_user_events(test_email_with_events, filter_pending=True)
print("⏰ Pending Events Test:")
print(f"📱 Pending events for notifications: {len(pending_events['events'])}")

# Show structure for Android integration
print("\n📱 JSON Response for Android (Correct Structure):")
sample_event = all_events['events'][0] if all_events['events'] else {}
print("Sample event structure:")
print(json.dumps(sample_event, indent=2))